In [12]:
import numpy as np
import xgboost as xgb
import optuna
from optuna.visualization import plot_optimization_history
import pandas as pd 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("../data/processed/all_shots_final-v2.csv", index_col=0)
df.drop(['PLAYER1_NAME'], axis=1, inplace=True)
df.dropna(inplace=True)
df.columns

Index(['Shot Distance', 'Season Type', 'Shot Zone Basic_In The Paint (Non-RA)',
       'Shot Zone Basic_Right Corner 3', 'Shot Zone Area_Right Side(R)',
       'Shot Zone Range_8-16 ft.', 'at_home', 'PREVIOUS_OFF_MISSED', 'Age',
       'ASTM', 'ORBM', 'FT%', 'height', 'weight', 'C', 'SG-PG', 'E_DEF_RATING',
       'PCT_AREA', 'DETAILLED_SHOT_TYPE_JUMP SHOT', 'target'],
      dtype='object')

## Séparation des données

In [6]:
# prepare data for modeling   
X = df.drop('target', axis = 1)
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, shuffle=True)

## Standardisation et réduction de dimension

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

df_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
df_scaled['target'] = y_train

# réduction de dimensions
sel_var = VarianceThreshold(0.0001)
X_train_scaled = sel_var.fit_transform(X_train_scaled, y_train)
X_test_scaled = sel_var.transform(X_test_scaled)

## XGBoost

In [11]:
train = xgb.DMatrix(data=X_train_scaled, label=y_train)
test = xgb.DMatrix(data=X_test_scaled, label=y_test)


def objective(trial):
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, train)
    preds = bst.predict(test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy

# Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-09-16 19:00:17,230] A new study created in memory with name: no-name-9b889ee0-0066-473d-b487-23e32383c27b
[I 2024-09-16 19:00:18,112] Trial 0 finished with value: 0.5645644595240594 and parameters: {'booster': 'dart', 'lambda': 0.15557153652820127, 'alpha': 2.6755285060640746e-05, 'subsample': 0.602952368420502, 'colsample_bytree': 0.8826107427536958, 'max_depth': 5, 'min_child_weight': 10, 'eta': 1.7336696389530763e-08, 'gamma': 2.35501581163008e-05, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.2145676327389237, 'skip_drop': 0.0016388768254819736}. Best is trial 0 with value: 0.5645644595240594.
[I 2024-09-16 19:00:18,695] Trial 1 finished with value: 0.5645644595240594 and parameters: {'booster': 'dart', 'lambda': 0.19858625246627276, 'alpha': 0.00115501571377454, 'subsample': 0.26357344119025666, 'colsample_bytree': 0.9340508917273367, 'max_depth': 3, 'min_child_weight': 4, 'eta': 0.00016896121107245156, 'gamma': 0.0404005

Number of finished trials:  200
Best trial:
  Value: 0.6844707171756037
  Params: 
    booster: dart
    lambda: 0.6737870636722346
    alpha: 0.05945403589338685
    subsample: 0.9391927120384811
    colsample_bytree: 0.751051549571555
    max_depth: 7
    min_child_weight: 2
    eta: 0.32294058008813725
    gamma: 0.4399138601349001
    grow_policy: lossguide
    sample_type: uniform
    normalize_type: tree
    rate_drop: 6.847639589409785e-07
    skip_drop: 0.0007494399472731043


In [13]:
plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                    50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                    66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
                    82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                    98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
                    111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
                    123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
                    135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
                    147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
                    159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
                    171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
                    183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
                    195, 196, 197, 198, 199],
              'y': [0.5645644595240594, 0.5645644595240594, 0.5645644595240594,
                    0.6110508703813822, 0.5645644595240594, 0.6746301024869705,
                    0.5645644595240594, 0.6716336119952896, 0.5645644595240594,
                    0.5645644595240594, 0.6745484860144344, 0.6745368265183579,
                    0.6746650809752003, 0.6745368265183579, 0.6745135075262048,
                    0.6748049949281192, 0.6745951239987408, 0.6743852530693624,
                    0.6745834645026642, 0.679690323784206, 0.6791889654529131,
                    0.674979887369268, 0.6774633600335793, 0.6766005573239124,
                    0.6726829666421817, 0.6595427145638766, 0.676670514300372,
                    0.6773234460806604, 0.5645644595240594, 0.6764373243788403,
                    0.5645644595240594, 0.679527090839134, 0.6768337472454441,
                    0.5645644595240594, 0.6790373920039176, 0.6798885352175079,
                    0.5723413434071379, 0.5645644595240594, 0.6806580619585622,
                    0.5645644595240594, 0.6811477607937785, 0.6823720078818194,
                    0.6797835997528187, 0.6807280189350217, 0.566931337227605,
                    0.6819289470309093, 0.5645644595240594, 0.5645644595240594,
                    0.6838644233796215, 0.5645644595240594, 0.6198887684074295,
                    0.6817890330779903, 0.682325369897513, 0.682465283850432,
                    0.6825702193151212, 0.6818589900544498, 0.6460176991150443,
                    0.5645644595240594, 0.6830599181503375, 0.6823486888896662,
                    0.682022222999522, 0.5645644595240594, 0.6752480557790292,
                    0.6823137104014364, 0.6730327515244792, 0.6822204344328238,
                    0.5645644595240594, 0.5645644595240594, 0.6721116513344293,
                    0.6811827392820082, 0.6762740914337683, 0.6829316636934951,
                    0.6823836673778959, 0.6804715100213369, 0.672869518579407,
                    0.6797486212645889, 0.639814847202304, 0.6818939685426796,
                    0.5842223699091725, 0.6748866114006553, 0.6805414669977964,
                    0.6827334522601933, 0.680331596068418, 0.6825702193151212,
                    0.6801217251390395, 0.6823486888896662, 0.5645644595240594,
                    0.6711322536639966, 0.674560145510511, 0.6821504774563644,
                    0.6817423950936841, 0.6823020509053599, 0.6826168572994276,
                    0.6811477607937785, 0.5645644595240594, 0.6827101332680402,
                    0.6830249396621078, 0.5645644595240594, 0.6829433231895717,
                    0.6817190761015309, 0.681800692574067, 0.6824

## Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
from optuna.samplers import TPESampler


def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    criterion = trial.suggest_categorical('criterion', ["gini", "entropy", "log_loss"])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        criterion=criterion
    )

    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# callback to stop the trial early 
pruning_callback = optuna.pruners.MedianPruner()

# Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize', pruner=[pruning_callback], sampler=TPESampler())
study.optimize(objective_rf, n_trials=30)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-09-16 21:16:25,101] A new study created in memory with name: no-name-3765e719-977e-47a1-be91-7ad6952a14ef
[I 2024-09-16 21:17:27,253] Trial 0 finished with value: 0.6821621369524409 and parameters: {'n_estimators': 212, 'max_depth': 27, 'min_samples_split': 23, 'min_samples_leaf': 21, 'max_features': 'sqrt', 'criterion': 'gini'}. Best is trial 0 with value: 0.6821621369524409.
[I 2024-09-16 21:18:29,193] Trial 1 finished with value: 0.6774633600335793 and parameters: {'n_estimators': 455, 'max_depth': 5, 'min_samples_split': 18, 'min_samples_leaf': 14, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 0 with value: 0.6821621369524409.
[I 2024-09-16 21:20:10,464] Trial 2 finished with value: 0.683048258654261 and parameters: {'n_estimators': 380, 'max_depth': 15, 'min_samples_split': 21, 'min_samples_leaf': 32, 'max_features': 'sqrt', 'criterion': 'entropy'}. Best is trial 2 with value: 0.683048258654261.
[I 2024-09-16 21:22:06,510] Trial 3 finished with value: 0.67

Number of finished trials:  30
Best trial:
  Value: 0.6835379574894773
  Params: 
    n_estimators: 333
    max_depth: 13
    min_samples_split: 3
    min_samples_leaf: 17
    max_features: log2
    criterion: entropy


In [20]:
plot_optimization_history(study)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
              'y': [0.6821621369524409, 0.6774633600335793, 0.683048258654261,
                    0.67112059416792, 0.6752480557790292, 0.6829433231895717,
                    0.6773467650728136, 0.6770202991826694, 0.6823486888896662,
                    0.6831998321032565, 0.676309069921998, 0.6831998321032565,
                    0.6829433231895717, 0.6821971154406706, 0.6815675026525354,
                    0.6819406065269858, 0.6804481910291837, 0.6831182156307204,
                    0.6830832371424908, 0.6798185782410484, 0.6829316636934951,
                    0.6835146384973242, 0.6833980435365583, 0.6829783016778015,
                    0.670817447269929, 0.6826051978033509, 0.6835379574894773,
                    0.6828150687327293, 0.6796553452959763, 0.6821388179602877]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
              'y': [0.6821621369524409, 0.6821621369524409, 0.683048258654261,
                    0.683048258654261, 0.683048258654261, 0.683048258654261,
                    0.683048258654261, 0.683048258654261, 0.683048258654261,
                    0.6831998321032565, 0.6831998321032565, 0.6831998321032565,
                    0.6831998321032565, 0.6831998321032565, 0.6831998321032565,
                    0.6831998321032565, 0.6831998321032565, 0.6831998321032565,
                    0.6831998321032565, 0.6831998321032565, 0.6831998321032565,
                    0.6835146384973242, 0.6835146384973242, 0.6835146384973242,
                    0.6835146384973242, 0.6835146384973242, 0.6835379574894773,
                    0.6835379574894773, 0.6835379574894773, 0.6835379574894773]},
             {'marker': {'color': '#cccccc'},
              'mode': 'markers',
              'name': 'Infeasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Optimization History Plot'},
               'xaxis': {'title': {'text': 'Trial'}},
               'yaxis': {'title': {'text': 'Objective Value'}}}
})

In [21]:
optuna.visualization.plot_param_importances(study)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'cliponaxis': False,
              'hovertemplate': [criterion (CategoricalDistribution):
                                0.009252275713017207<extra></extra>, max_features
                                (CategoricalDistribution):
                                0.016860524205585647<extra></extra>, n_estimators
                                (IntDistribution):
                                0.11753735559645057<extra></extra>,
                                min_samples_split (IntDistribution):
                                0.13949586459615063<extra></extra>,
                                min_samples_leaf (IntDistribution):
                                0.19009482563751487<extra></extra>, max_depth
                                (IntDistribution):
                                0.5267591542512812<extra></extra>],
              'name': 'Objective Value',
              'orientation': 'h',
              'text': [<0.01, 0.02, 0.12, 0.14, 0.19, 0.53],
              'textposition': 'outside',
              'type': 'bar',
              'x': [0.009252275713017207, 0.016860524205585647,
                    0.11753735559645057, 0.13949586459615063, 0.19009482563751487,
                    0.5267591542512812],
              'y': [criterion, max_features, n_estimators, min_samples_split,
                    min_samples_leaf, max_depth]}],
    'layout': {'template': '...',
               'title': {'text': 'Hyperparameter Importances'},
               'xaxis': {'title': {'text': 'Hyperparameter Importance'}},
               'yaxis': {'title': {'text': 'Hyperparameter'}}}
})

In [22]:
optuna.visualization.plot_slice(study, params = ['n_estimators',
'max_depth',
'min_samples_split',
'min_samples_leaf',
'max_features',
'criterion',])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                                   26, 27, 28, 29],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5},
                         'showscale': True},
              'mode': 'markers',
              'name': 'Feasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [gini, entropy, entropy, log_loss, log_loss, entropy, log_loss,
                    entropy, log_loss, log_loss, gini, entropy, log_loss, entropy,
                    gini, log_loss, entropy, entropy, log_loss, gini, log_loss,
                    entropy, entropy, entropy, entropy, entropy, entropy, entropy,
                    entropy, gini],
              'xaxis': 'x',
              'y': [0.6821621369524409, 0.6774633600335793, 0.683048258654261,
                    0.67112059416792, 0.6752480557790292, 0.6829433231895717,
                    0.6773467650728136, 0.6770202991826694, 0.6823486888896662,
                    0.6831998321032565, 0.676309069921998, 0.6831998321032565,
                    0.6829433231895717, 0.6821971154406706, 0.6815675026525354,
                    0.6819406065269858, 0.6804481910291837, 0.6831182156307204,
                    0.6830832371424908, 0.6798185782410484, 0.6829316636934951,
                    0.6835146384973242, 0.6833980435365583, 0.6829783016778015,
                    0.670817447269929, 0.6826051978033509, 0.6835379574894773,
                    0.6828150687327293, 0.6796553452959763, 0.6821388179602877],
              'yaxis': 'y'},
             {'marker': {'color': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                                   26, 27, 28, 29],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5},
                         'showscale': False},
              'mode': 'markers',
              'name': 'Feasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [27, 5, 15, 25, 32, 15, 26, 22, 10, 13, 4, 15, 12, 19, 9, 19,
                    8, 14, 17, 7, 12, 13, 12, 11, 2, 19, 13, 17, 7, 21],
              'xaxis': 'x2',
              'y': [0.6821621369524409, 0.6774633600335793, 0.683048258654261,
                    0.67112059416792, 0.6752480557790292, 0.6829433231895717,
                    0.6773467650728136, 0.6770202991826694, 0.6823486888896662,
                    0.6831998321032565, 0.676309069921998, 0.6831998321032565,
                    0.68294